## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from Config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..\Weather_Database\WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Piet Retief,-27.0071,30.8132,66.63,62,0,5.73,ZA,2022-03-31 15:46:15,clear sky
1,1,Komsomolskiy,40.4272,71.7189,56.12,58,0,4.61,UZ,2022-03-31 15:46:15,clear sky
2,2,Hilo,19.7297,-155.0900,69.53,91,75,5.75,US,2022-03-31 15:43:52,broken clouds
3,3,Kapaa,22.0752,-159.3190,73.11,95,20,11.50,US,2022-03-31 15:42:58,few clouds
4,4,Suntar,62.1444,117.6319,30.06,97,100,7.90,RU,2022-03-31 15:46:16,light snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
3,3,Kapaa,22.0752,-159.3190,73.11,95,20,11.50,US,2022-03-31 15:42:58,few clouds
5,5,Avarua,-21.2078,-159.7750,75.25,94,6,2.30,CK,2022-03-31 15:46:16,clear sky
9,9,Gobabis,-22.4500,18.9667,77.50,47,95,13.29,NaN,2022-03-31 15:43:24,overcast clouds
16,16,Puerto Ayora,-0.7393,-90.3518,82.36,75,94,6.98,EC,2022-03-31 15:46:18,light rain
18,18,Bolobo,-2.1667,16.2333,88.83,50,49,3.04,CD,2022-03-31 15:46:19,scattered clouds
20,20,Morondava,-20.2833,44.2833,84.04,67,29,6.53,MG,2022-03-31 15:46:19,scattered clouds
22,22,Katubao,5.8892,124.8294,79.77,81,64,2.59,PH,2022-03-31 15:46:20,broken clouds
23,23,Busselton,-33.6500,115.3333,72.72,66,0,12.53,AU,2022-03-31 15:46:20,clear sky
24,24,Tonantins,-2.8731,-67.8022,83.12,68,94,4.70,BR,2022-03-31 15:46:20,overcast clouds
32,32,Abha,18.2164,42.5053,73.22,49,20,10.36,SA,2022-03-31 15:46:23,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                236
City                   236
Lat                    236
Lng                    236
Max Temp               236
Humidity               236
Cloudiness             236
Wind Speed             236
Country                231
Date                   236
Current Description    236
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)].dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kapaa,US,73.11,few clouds,22.0752,-159.3190,
5,Avarua,CK,75.25,clear sky,-21.2078,-159.7750,
16,Puerto Ayora,EC,82.36,light rain,-0.7393,-90.3518,
18,Bolobo,CD,88.83,scattered clouds,-2.1667,16.2333,
20,Morondava,MG,84.04,scattered clouds,-20.2833,44.2833,
22,Katubao,PH,79.77,broken clouds,5.8892,124.8294,
23,Busselton,AU,72.72,clear sky,-33.6500,115.3333,
24,Tonantins,BR,83.12,overcast clouds,-2.8731,-67.8022,
32,Abha,SA,73.22,few clouds,18.2164,42.5053,
34,Ambulu,ID,76.35,light rain,-8.3450,113.6058,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 10000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotel = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
import numpy as np

In [10]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df["Hotel Name"] = hotel_df["Hotel Name"].replace('',np.nan,regex = True)
clean_hotel_df = hotel_df.dropna()

#hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"].dropna(inplace = True)

In [11]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kapaa,US,73.11,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
5,Avarua,CK,75.25,clear sky,-21.2078,-159.7750,Little Polynesian Resort
16,Puerto Ayora,EC,82.36,light rain,-0.7393,-90.3518,Finch Bay Galapagos Hotel
18,Bolobo,CD,88.83,scattered clouds,-2.1667,16.2333,Hotel de Stephen
20,Morondava,MG,84.04,scattered clouds,-20.2833,44.2833,La Case Bambou
...,...,...,...,...,...,...,...
647,Kudahuvadhoo,MV,83.28,clear sky,2.6708,72.8944,Niyama Private Islands Maldives
648,Noumea,NC,82.99,clear sky,-22.2763,166.4572,Hôtel Le Lagon
649,Mangan,IN,71.28,overcast clouds,27.5167,88.5333,Munlom Nature Resort
650,Bacolod,PH,79.52,few clouds,10.6667,122.9500,Planta Hotel & Residences


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))